# Quantum Absorption Estimation for Saturable Samples #

Author: Jake Biele (jb12101@bristol.ac.uk)
Date: 09/12/21

### Summary ###

This notebook outlines the code used to model the results presented in the paper DOI: 10.1103/PhysRevA.00.003700



In [1]:
# List of required packages

import numpy as np

from qutip import *
# see https://qutip.org/ for installation guide

import math
from scipy.special import wrightomega, lambertw

### Defined Functions

In [ ]:

def Transmission(L, a, Nin, Ns):
    
    '''Function takes L = length [cm], a = linear absorption coefficient [1/cm]
    Nin = Input Intensity [W/cm^2] and Ns = saturation intensity [W/cm^2]
    
    and returns the sample transmission.'''
    
    return (wrightomega(np.log(2*Nin/Ns)+((2*Nin)/Ns) - a*L)*Ns)/(2*Nin)


def quantumBound(L, a, Nin, Ns):
    
    '''Function takes L = length [cm], a = linear absorption coefficient [1/cm]
    Nin = Input Intensity [W/cm^2] and Ns = saturation intensity [W/cm^2]
    
    and returns the quantum bound on precision.'''
    
    kappa = 2*Nin/Ns
    eta = (1/kappa)*wrightomega(np.log(kappa)+kappa-L*a)
    
    return np.real((eta/(1-eta))*(L/(1+kappa*eta))**2)*Nin

def classicalBound(L, a, Nin, Ns):
    
    '''Function takes L = length [cm], a = linear absorption coefficient [1/cm]
    Nin = Input Intensity [W/cm^2] and Ns = saturation intensity [W/cm^2]
    
    and returns the classical bound on precision.'''
    
    kappa = 2*Nin/Ns
    eta = (1/kappa)*wrightomega(np.log(kappa)+kappa-L*a)
    
    return np.real((eta)*(L/(1+kappa*eta))**2)*Nin

### FULLY QUANTUM MODEL ###

def full_model(state, n, na, alpha, spr, dpr, deltaz, Ntrunc, l):
    
    '''Function takes a QuTip defined quantum state, n = number of photons in the state, alpha = linear abosrption, spr and dpr =
    the dexcitaion and dephasing rates, deltaz = z discritisation, Ntrunc = Fock space truncation for QuTip state.
    
    returns a list of times at which the state reaches each step along z and a list of accociated transmission and variance
    expactation values used to calculate the Fano factor
    
    NOTE this function calls the propagator function below.'''
    
    nn = []
    nn.append(n)
    varss = []
    
    # initialise state in ket form
    if state == 'fock':

        psi0 = fock_dm(Ntrunc, n)
        varss.append(0)
    elif state == 'coherent':
        
        psi0 = coherent_dm(Ntrunc, np.sqrt(n))
        varss.append(n)
    steps = int(l/deltaz)
    
    for m in range(steps):
        
        ts, _, v, nout, tot, _, fano, psi0, fisher = propagator(psi0, n, na, alpha, spr, dpr, deltaz, Ntrunc)

        nn.append(nout[-1])
        varss.append(v[-1])
        
    trans = [k/n for k in nn]
    times = np.linspace(0, l, l/deltaz+1)
    
    return times, trans, varss

def propagator(psi0, n, na, sigma, spr, dpr, l, Ntrunc):
    
    
    # field operator + enviroment hilbert space
    ahs = [qeye(2) for _ in range(na)]
    ahs.insert(0, destroy(Ntrunc))
    a = tensor(ahs)
    
    # field hilbert space + kth absorbing operator
    kth_cross_op_list, op_list, coll_list = [], [], []
    lis = np.linspace(1, na, na)
    
    for k in lis:
        
        ahs = [qeye(2) for _ in range(na)]
        ahs[int(k)-1] = destroy(2).dag()
        ahs.insert(0, qeye(Ntrunc))
        op = (a * tensor(ahs) + a.dag() * tensor(ahs).dag())
        kth_cross_op_list.append(op)
        
        op = tensor(ahs) * tensor(ahs).dag()
        op_list.append(op)
        
        kth_coll = np.sqrt(spr)*tensor(ahs).dag()
        coll_list.append(kth_coll)  
        
        ahs = [qeye(2) for _ in range(na)]
        ahs[int(k)-1] = sigmaz()
        ahs.insert(0, qeye(Ntrunc))
        dth_coll = np.sqrt(dpr)*tensor(ahs)
        coll_list.append(dth_coll)
        
    # Hamiltonian
    H_field = a.dag() * a + np.sum(op_list) + sigma * np.sum(kth_cross_op_list)
    
    # Number operator
    num_op = a.dag() * a
    
    # Timeline
    times = np.linspace(0,l,1000)
    
    ahs = [fock_dm(2,0) for _ in range(na)]
    ahs.insert(0, psi0.ptrace(0))
    state = tensor(ahs)

    # Evolve master equation
    result = mesolve(H_field, state, times, coll_list, [])

    NN = expect(num_op**2, result.states)
    N = expect(num_op, result.states)
    J = expect(np.sum(op_list), result.states)
    
    # Analytics on output state
    var = [nn-r**2 for nn, r in zip(NN, N)]
    trans = N/n
    nout = N
    tot = N+J
    natom = J
    fano = var/nout
    fs = result.states[-1]
    fisher = [n**2/v for v in var]
            
    return times, trans, var, nout, tot, natom, fano, fs, fisher

### Quantum and Classical Fisher Information ###

In [ ]:

# Define saturation intensity, porbe intensity range and linear absorptions to calculate
Ns = 4
Nin = np.linspace(1,20,200)
linAb = [0.5, 1, 2]

# Calculate kappa
kappa = [n*2/Ns for n in Nin]

# Initiate lists to handle data. Data format: [[x],[y1],[y2]...]
TFI_quantum, TFI_classical = [[kappa]], [[kappa]]
FI_quantum, FI_classical = [[kappa]], [[kappa]]
quantum_advantage = [[kappa]]

# Calculate Total FI and FI per input power
for a in linAb:
    
    # Quantum bound
    TFI_quantum.append([quantumBound(n, Ns,a) for n in Nin])
    FI_quantum.append([quantumBound(n, Ns,a)/n for n in Nin])
    
    # Classical bound
    TFI_classical.append([classicalBound(n, Ns, a) for n in Nin])
    FI_classical.append([quantumBound(n, Ns,a)/n for n in Nin])
    
    # Quantum advantage
    quantum_advantage.append([quantumBound(n, Ns, a)/classicalBound(n, Ns, 0.5) for n in Nin])


### Quantum Loss Channel ###

In [ ]:
# Define variable

deltaz = 1
l = 20
n = 12
na = 4
alpha = 0.4
beta = 5

spr = alpha/tau_int
g = np.sqrt(3/(2*alpha))*(3/(np.pi**2*beta))*2*np.pi
dpr = 4

In [ ]:
# Coherent state propagation

lins = np.linspace(0,l,l/deltaz+1)

Nin = [n]

fock_data = []
coherent_data = []

for i, n in enumerate(Nin):
    
    times, trans, varss = full_model(state = 'coherent', n = int(n), na = na, deltaz = deltaz, alpha = g, spr = spr, dpr = dpr, l = l, Ntrunc = 3*n)
    coherent_data.append(varss)
        
coherent_data.insert(0, times.tolist())

# Classical fano
fano_c = [t*n/(t*n) for t in trans]

# Quantum fano
fano_q = [v/(t*n) for v, t in zip(varss, trans)]


In [ ]:
# Fock state propagation

lins = np.linspace(0,l,l/deltaz+1)

Nin = [n]

fock_data = []

for i, n in enumerate(Nin):
    
    times, trans, varss = full_model(state = 'fock', n = int(n), na = na, deltaz = deltaz, alpha = g, spr = spr, dpr = dpr, l = l, Ntrunc = n+1)
    fock_data.append(varss)
        
fock_data.insert(0, times.tolist())

# Classical fano
fano_c = [t*(1-t)/t for t in trans]

# Quantum fano
fano_q = [v/(t*n) for v, t in zip(varss, trans)]
